In [2]:
from __future__ import annotations
import json
import pandas as pd
import json
import numpy as np
from typing import Iterable
from dataclasses import dataclass, field

# http://etetoolkit.org/download/
# http://etetoolkit.org/docs/latest/tutorial/tutorial_ncbitaxonomy.html#setting-up-a-local-copy-of-the-ncbi-taxonomy-database
# warning: ete tk inserts itself into ~/.local/share/ete/
from ete4 import NCBITaxa
ncbi = NCBITaxa()

from local.utils import regex
from local.web import ncbi_search, chain_get


In [3]:
_df = pd.read_csv("./cache/ecoli_nr.tsv", sep="\t", header=None)
hits = {}
for _, row in _df.iterrows():
    query, subject, pident, bitscore = row
    hits[query] = hits.get(query, []) + [(subject, pident, bitscore)]

for q, d in hits.items():
    hits[q] = sorted(d, key=lambda x: x[2], reverse=True)

In [6]:
tax_id_hits: dict[str, int] = {}
for i, (q, d) in enumerate(hits.items()):
    print(f"{i+1}/{len(hits)}", end="\r")
    best = d[0]
    subject, pident, bitscore = best

    err, all_meta = ncbi_search(subject, "protein", silent=True)
    if err is not None: continue
    meta = chain_get(all_meta[0], "eSummaryResult, DocSum, Item")
    if meta is None: continue
    tax = [x for x in meta if x["@Name"] == "TaxId"][0]["#text"]
    tax_id_hits[tax] = tax_id_hits.get(tax, 0) + 1